In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import TargetEncoder, LabelEncoder, OrdinalEncoder, MinMaxScaler, OneHotEncoder, power_transform, PowerTransformer
import category_encoders as ce
import pickle
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import dill
import os
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier
import imblearn
from imblearn.under_sampling import RandomUnderSampler

In [2]:
with open('read_file.pickle', 'rb') as f: 
    load_data = dill.load(f)


df = load_data('application_train.csv')

In [3]:
with open('clean_data.pickle', 'rb') as f:
    clean_data = dill.load(f)

df = clean_data(df)

In [4]:
df.isnull().sum().sum()

np.int64(0)

In [13]:
df.TARGET.value_counts() # imbalanced data

TARGET
0    282686
1     24825
Name: count, dtype: int64

In [5]:
with open('encode_data.pickle', 'rb') as f:
    encode_data = dill.load(f)


# Get only categorical features:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

df = encode_data(df, 'TARGET', categorical_cols, train=True)


In [6]:
df

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0.083459,0.101419,0.085002,0.079616,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0.083459,0.069993,0.085002,0.083249,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,0.054784,0.101419,0.072437,0.079616,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0.083459,0.069993,0.085002,0.079616,0,135000.0,312682.5,29686.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
4,100007,0,0.083459,0.101419,0.085002,0.079616,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0.083459,0.101419,0.085002,0.083249,0,157500.0,254700.0,27558.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
307507,456252,0,0.083459,0.069993,0.085002,0.079616,0,72000.0,269550.0,12001.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
307508,456253,0,0.083459,0.069993,0.085002,0.079616,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,0.083459,0.069993,0.085002,0.079616,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
with open('scale_data.pickle', 'rb') as f:
    scale_data = dill.load(f)


df = scale_data(df, target ='TARGET', features_to_scale=None)

In [8]:
df

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,-1.733423,1,0.324395,1.387977,0.717914,-0.664531,-0.577538,0.142129,-0.478095,-0.166143,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-0.440926
1,-1.733413,0,0.324395,-0.720174,0.717914,1.504820,-0.577538,0.426792,1.725450,0.592683,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-1.007331
2,-1.733403,0,-3.082659,1.387977,-1.392925,-0.664531,-0.577538,-0.427196,-1.152888,-1.404669,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-1.007331
3,-1.733384,0,0.324395,-0.720174,0.717914,-0.664531,-0.577538,-0.142533,-0.711430,0.177874,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-0.440926
4,-1.733374,0,0.324395,1.387977,0.717914,-0.664531,-0.577538,-0.199466,-0.213734,-0.361749,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-1.007331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,1.732372,0,0.324395,1.387977,0.717914,1.504820,-0.577538,-0.047646,-0.855489,0.031015,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-0.440926
307507,1.732381,0,0.324395,-0.720174,0.717914,-0.664531,-0.577538,-0.408219,-0.818594,-1.042333,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-0.440926
307508,1.732391,0,0.324395,-0.720174,0.717914,-0.664531,-0.577538,-0.066623,0.195379,0.198056,...,-0.090534,-0.024402,-0.022529,-0.018305,12.747224,-0.058766,-0.155837,0.897175,-0.30862,-0.440926
307509,1.732401,1,0.324395,-0.720174,0.717914,-0.664531,-0.577538,0.009287,-0.568757,-0.476318,...,-0.090534,-0.024402,-0.022529,-0.018305,-0.070987,-0.058766,-0.155837,-0.269947,-0.30862,-1.007331


In [14]:
with open('split_data.pickle', 'rb') as f:
    split_data = dill.load(f)


X_train, X_test, y_train, y_test = split_data(df, 'TARGET', feature_selected= None, features_dropped =['SK_ID_CURR'], balanced_data=False)

In [15]:
with open('train_model.pickle', 'rb') as f:
    train_model = dill.load(f)

train_model = train_model(LogisticRegression, xtrain=X_train, ytrain=y_train) 

with open('trained_model.pickle', 'wb') as f: 
    dill.dump(train_model, f)

c:\Users\elige\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
